In [13]:
import requests
import datetime
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML

/var/folders/85/178rxd5d5b31szqc8b3mznd40000gn/T/ipykernel_27755/1126331791.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [14]:
display(HTML("<style>.container { width:90% !important; }</style>"))

# 한국은행 기준금리

In [3]:
base_url= "https://www.bok.or.kr/portal/singl/baseRate/list.do?dataSeCd=01&menuNo=200643"
resp = requests.get(base_url, 'html.parser')
soup= BeautifulSoup(resp.text, 'lxml')

In [4]:
dict_list= []

for i in range(45):
    dict_list.append({
        'DATE': pd.to_datetime(datetime.datetime.strptime(soup.find_all('td')[3*i].text + soup.find_all('td')[3*i+1].text, '%Y%m월 %d일')),
        'RATE': float(soup.find_all('td')[3*i+2].text)
    })

In [5]:
pd_result= pd.DataFrame(dict_list)

# 원하는 전체 date range
idx= pd.date_range(pd_result.DATE.min(), pd_result.DATE.max())
idx= pd.Series(idx)

In [25]:
# 없는 행에 Nan을 넣어 병합한 후 Nan값을 이전 값으로 메꾸기
baserate= pd.concat([pd.DataFrame({'DATE': idx[~idx.isin(pd_result.DATE)], 'RATE': np.nan}),pd_result]).sort_values('DATE').reset_index(drop=True).ffill(axis=0)

In [29]:
baserate

,DATE,RATE
0,1999-05-06,4.75
1,1999-05-07,4.75
2,1999-05-08,4.75
3,1999-05-09,4.75
4,1999-05-10,4.75
...,...,...
7689,2020-05-24,0.75
7690,2020-05-25,0.75
7691,2020-05-26,0.75
7692,2020-05-27,0.75


In [26]:
baserate.to_csv('./baserate.csv')

# 콜금리

In [15]:
base_url2= "https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL&page={}"

In [16]:
page_num= 384

In [17]:
soup_list2= []

for i in range(1,page_num+1):
    soup_list2.append(BeautifulSoup(requests.get(base_url2.format(i), 'html.parser').text,'lxml').find_all('td'))

In [18]:
result_list2= []

for lists in soup_list2:
    for i in range(len(lists)//4):
        result_list2.append({
            'date': lists[4*i].text.strip(),
            'rate': float(lists[4*i+1].text)
        })

In [19]:
pd_result2= pd.DataFrame(result_list2)
pd_result2.date= pd_result2.date.astype('datetime64')

In [20]:
# 원하는 전체 date range
idx2= pd.date_range(pd_result2.date.min(), pd_result2.date.max())
idx2= pd.Series(idx2)

In [21]:
# 없는 행에 Nan을 넣어 병합한 후 Nan값을 이전 값으로 메꾸기
callrate= pd.concat([pd.DataFrame({'date': idx2[~idx2.isin(pd_result2.date)], 'rate': np.nan}),pd_result2]).sort_values('date').reset_index(drop=True).ffill(axis=0)[1:]

In [22]:
callrate

,date,rate
1,2011-12-31,3.29
2,2012-01-01,3.29
3,2012-01-02,3.25
4,2012-01-03,3.25
5,2012-01-04,3.24
...,...,...
3972,2022-11-14,3.07
3973,2022-11-15,3.03
3974,2022-11-16,3.03
3975,2022-11-17,3.02


In [23]:
callrate.to_csv('../data/call_rate.csv')

In [24]:
file1 = pd.read_csv('../data/call_rate.csv')



In [25]:
file1

,Unnamed: 0,date,rate
0,1,2011-12-31,3.29
1,2,2012-01-01,3.29
2,3,2012-01-02,3.25
3,4,2012-01-03,3.25
4,5,2012-01-04,3.24
...,...,...,...
3971,3972,2022-11-14,3.07
3972,3973,2022-11-15,3.03
3973,3974,2022-11-16,3.03
3974,3975,2022-11-17,3.02


In [ ]:
file1.style.hide_index()

In [27]:
file2 = file1.drop(['Unnamed: 0'], axis=1)

In [28]:
file2

file1.style.hide_index()

,date,rate
0,2011-12-31,3.29
1,2012-01-01,3.29
2,2012-01-02,3.25
3,2012-01-03,3.25
4,2012-01-04,3.24
...,...,...
3971,2022-11-14,3.07
3972,2022-11-15,3.03
3973,2022-11-16,3.03
3974,2022-11-17,3.02


In [29]:
file2.to_csv('../data/call_rate.csv')

In [ ]:
callrate['shift'] = callrate['RATE'].shift(periods=30, fill_value=0)

In [ ]:
callrate

In [ ]:
conditions = [
    (callrate['RATE'] - callrate['shift']) > 0,
    (callrate['RATE'] - callrate['shift']) < 0
]

choices = ['up','down']

callrate['label'] = np.select(conditions, choices, default='neutral')

# %%
callrate

# %%
callrate['label'].value_counts()

# %%
callrate.to_csv('./labeled_callrate.csv')